In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [3]:
def rho_shock_tube(x, rhoL, rhoR):
    rho = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    rho[mask1] = rhoL
    rho[mask2] = rhoR
    return rho

In [4]:
def u_shock_tube(x, uL, uR):
    u = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    u[mask1] = uL
    u[mask2] = uR
    return u

In [5]:
def P_shock_tube(x, PL, PR):
    P = numpy.zeros_like(x)
    mask1 = numpy.where(x < 0.0)
    mask2 = numpy.where(0.0 <= x)
    P[mask1] = PL
    P[mask2] = PR
    return P

In [6]:
nx = 81
L = 10.0
dx = 0.25
dt = 0.0002
gamma = 1.4
rhoL = 1.0
rhoR = 0.125
uL = 0.0
uR = 0.0
PL = 100.0
PR = 10.0
nt = 50

x = numpy.linspace(-L, L, num=nx)

rho0 = rho_shock_tube(x, rhoL, rhoR)
u0 = u_shock_tube(x, uL, uR)
P0 = P_shock_tube(x, PL, PR)

In [7]:
def e_def(P, rho, gamma):
    e = P / ((gamma - 1) * rho)
    return e

def U_def(rho, u, P, e):
    U = numpy.array([rho.copy(),
                     u.copy() * rho.copy(),
                     P.copy() * (e * 0.5 * u.copy()**2)])
    return U

In [8]:
def U_def(rho, u):
    U = numpy.array([u.copy() * rho.copy()])
    return U

In [9]:
e0 = e_def(P0, rho0, gamma)

U0 = U_def(rho0, u0, P0, e0)

In [10]:
U0 = U_def(rho0, u0)

In [11]:
U0

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]])

def flux(U, gamma, rho, u, P, e, *args):
    F = numpy.zeros_like(U)
    F[0] = U[1]
    F[1] = U[1]**2 / U[0] + (gamma - 1) * (U[2] - 0.5 * U[1]**2 / U[0])
    F[2] = (U[2] + (gamma - 1) * (U[2] - 0.5 * U[1]**2 / U[0])) * U[1] / U[0]
    return F

In [12]:
def flux(U, u, P):
    F = U
    return F

def rich(U0, nt, gamma, dt, dx, bc_values, rho, u, P, e, *args):
    U_hist = [U0.copy()]
    U = U0.copy()
    U_half = U.copy()
    for n in range(nt):
        F = flux(U, gamma, rho, u, P, e, *args)
        U_half[:,1:-1] = 0.5 * (U[:,2:] + U[:,:-2]) - dt / 2*dx * (F[:,2:] - F[:,:-2])
        F = flux(U_half, gamma, rho, u, P, e, *args)
        U[:,1:-1] = U[:,1:-1] - dt / dx * (F[:,2:] - F[:,:-2])
        U[:,0] = bc_values[0]
        U[:,-1] = bc_values[1]
        U_hist.append(U.copy())
    return U_hist

In [13]:
def minmod(e, dx):
    sigma = numpy.zeros_like(e)
    for i in range(1, len(e) - 1):
        de_minus = (e[i] - e[i - 1]) / dx
        de_plus = (e[i + 1] - e[i]) / dx
        if de_minus > 0 and de_plus > 0:
            sigma[i] = min(de_minus, de_plus)
        elif de_minus < 0 and de_plus < 0:
            sigma[i] = max(de_minus, de_plus)
        else:
            sigma[i] = 0.0
    return sigma

In [14]:
def muscl(U0, nt, gamma, dt, dx, bc_values, rho, u, P, e):
    def compute_flux(U, u, P):
        sigma = minmod(U, dx)
        UL = (U + sigma * dx / 2.0)[:-1]
        UR = (U - sigma * dx / 2.0)[1:]
        F = 0.5 * (flux(UL, u, P) + flux(UR, u, P) -
                   dx / dt * (UR - UL))
        return F
    U = U0.copy()
    U_half = U.copy()
    for n in range(nt):
        F = compute_flux(U, u, P)
        U_half[1:-1] = 0.5 * (U[2:] + U[:-2]) - dt / (2*dx) * (F[2:] - F[:-2])
        U_half[0], U_half[-1] = bc_values
        F = compute_flux(U_half, u, P)
        U[1:-1] = U[1:-1] - dt / dx * (F[2:] - F[:-2])
        U[0], U[-1] = bc_values
    return U

In [15]:
U = muscl(U0, nt, gamma, dt, dx, (U0[0], U0[-1]), rho0, u0, P0, e0)

In [16]:
U

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]])